In [38]:
from gensim.models import Word2Vec
import pandas
import numpy
import os

In [39]:
# 窗口大小和特征维度
min_count = int(2)
print(min_count)
dim = 15
window = 5

2


In [40]:
print('Distributed represntation will be learned based on vector dim: ' + str(dim) + ', context window: ' + str(window) + '.')
df = pandas.read_csv(os.path.join('train_test_data/train_data/proteins.txt'),delim_whitespace=True,header=0)
print(df)

Distributed represntation will be learned based on vector dim: 15, context window: 5.
           Peptide      HLA  BindingCategory
0       AAAAAAAALY  A*29:02                1
1        AAAAALQAK  A*03:01                1
2         AAAAALWL  C*16:01                1
3        AAAAARAAL  B*14:02               -1
4        AAAAEEEEE  A*02:01               -1
...            ...      ...              ...
141219   YYYNFSEDL  B*27:05               -1
141220   YYYNFSEDL  B*35:01               -1
141221   YYYNFSEDL  B*40:01               -1
141222   YYYNFSEDL  B*44:02               -1
141223   YYYNFSEDL  B*58:01               -1

[141224 rows x 3 columns]


In [41]:
df.columns = ['sequence','HLA','target']
print(df.columns)
print(df)

Index(['sequence', 'HLA', 'target'], dtype='object')
          sequence      HLA  target
0       AAAAAAAALY  A*29:02       1
1        AAAAALQAK  A*03:01       1
2         AAAAALWL  C*16:01       1
3        AAAAARAAL  B*14:02      -1
4        AAAAEEEEE  A*02:01      -1
...            ...      ...     ...
141219   YYYNFSEDL  B*27:05      -1
141220   YYYNFSEDL  B*35:01      -1
141221   YYYNFSEDL  B*40:01      -1
141222   YYYNFSEDL  B*44:02      -1
141223   YYYNFSEDL  B*58:01      -1

[141224 rows x 3 columns]


In [42]:
# 删除包含未知的氨基酸；例如：’X‘和’B‘
df = df[df.sequence.str.contains('X')==False]
print(df)
df = df[df.sequence.str.contains('B')==False]
print(df)

          sequence      HLA  target
0       AAAAAAAALY  A*29:02       1
1        AAAAALQAK  A*03:01       1
2         AAAAALWL  C*16:01       1
3        AAAAARAAL  B*14:02      -1
4        AAAAEEEEE  A*02:01      -1
...            ...      ...     ...
141219   YYYNFSEDL  B*27:05      -1
141220   YYYNFSEDL  B*35:01      -1
141221   YYYNFSEDL  B*40:01      -1
141222   YYYNFSEDL  B*44:02      -1
141223   YYYNFSEDL  B*58:01      -1

[141110 rows x 3 columns]
          sequence      HLA  target
0       AAAAAAAALY  A*29:02       1
1        AAAAALQAK  A*03:01       1
2         AAAAALWL  C*16:01       1
3        AAAAARAAL  B*14:02      -1
4        AAAAEEEEE  A*02:01      -1
...            ...      ...     ...
141219   YYYNFSEDL  B*27:05      -1
141220   YYYNFSEDL  B*35:01      -1
141221   YYYNFSEDL  B*40:01      -1
141222   YYYNFSEDL  B*44:02      -1
141223   YYYNFSEDL  B*58:01      -1

[141089 rows x 3 columns]


In [43]:
# 将训练数据打乱
df = df.sample(frac=1)
print(df)

         sequence      HLA  target
55628  KDCVMYASAL  B*40:01      -1
12139   CLINDPWVL  A*02:01       1
78532   MLGEETIKV  B*15:01      -1
16934  DTIESAKTKI  A*02:01      -1
42914   HFDDVANGF  A*02:01      -1
...           ...      ...     ...
45179   HTASGEHSL  A*01:01      -1
13809   DEGFHAATV  A*26:01      -1
91828   QPKPGTRMV  B*07:02       1
89851   QEILDLWVY  B*45:01      -1
54846   IYTTNDNNY  A*68:02       1

[141089 rows x 3 columns]


In [44]:

# 用word2vec提取肽序列为列表
text = list(df.sequence)
print(text)

df = []
sentences = []
for aa in range(len(text)):
    sentences.append(list(text[aa]))

print('There are '+str(len(sentences))+' peptide sequences in the source file...')


['KDCVMYASAL', 'CLINDPWVL', 'MLGEETIKV', 'DTIESAKTKI', 'HFDDVANGF', 'AFYWHFIFR', 'DTDISQLHH', 'PTLFGRGVI', 'DYMPSMKRFR', 'NLKDEQFPV', 'AIMDKNIML', 'RPRCAYLPF', 'AENCYNLEI', 'VSSHKGWAK', 'VYIGDPAQL', 'DSFLRKIGDK', 'KETLYRIDG', 'GEENFSSRMY', 'TPREAPYEL', 'VMETENALF', 'CWLVSNGSY', 'IVPFWITAIY', 'IPSTVKTNLY', 'DIIKILDKY', 'RPRFDDAYNI', 'FPYTGDPPY', 'TVANNPDDK', 'PEDPVEIALY', 'LLVPFVQWFV', 'YMRERFEPM', 'VEKYIEEQL', 'LMYFHRRDLR', 'REVGQLIGL', 'TFDVSPKAV', 'SVPEPAAGI', 'RAFLLRHYY', 'LFMSHVKSV', 'GSFRKICGF', 'FQVNRFTGY', 'LCSEKPVMHY', 'QAALQSAWQ', 'WLLIDTSNA', 'RFSWLSLLVPF', 'YVWWAAVIY', 'NFVRSSNLK', 'IYAGSLSAL', 'SYLKPHIFE', 'HSRAYQQAL', 'GIFCFRILL', 'AMKAQFERDL', 'WPVLGSEVL', 'LLLRPFWPA', 'AEVEEHRTV', 'YLLVKWIRK', 'IFMLQKCDL', 'KLWAQCVQL', 'TMRTPLFPW', 'NPRVKAAAL', 'FPAQPGLTSA', 'DENPYKTWAY', 'AAKKKGASL', 'TLVDICFWS', 'SPLQLRFRM', 'NRAVKPTGS', 'MSDIFASEV', 'FVFEATKLY', 'SAVNMTSRM', 'KEAYCQEFLL', 'EGFLKAAMF', 'TTIEDILPK', 'KAIIDTAQF', 'IQGNVTSIH', 'TFMYVFSTF', 'TESDAIRTL', 'VYAYPSGEK', 'RAMDV

In [46]:
model = Word2Vec(sentences,min_count=min_count,size=dim,window=window,sg=True,iter=10,batch_words=100)

d:\python3.7.7\lib\site-packages\gensim\models\word2vec.py:813: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


In [52]:
# 模型保存
model.save(os.path.join('HLA-Vec_embedding','HLA_Vec_model.model'))